In [ ]:
import requests
import pandas as pd
import json


<h1>tournament</h1>

In [20]:
import requests
import json
import pandas as pd

url_graphql = "https://api-op.grid.gg/central-data/graphql"
headers1 = {
    "x-api-key": "gDVqIdMead2zTW1DKehu8PicvVXStT2xtmbYHK7b", 
    "Content-Type": "application/json",
    "Accept": "application/json",
}

querytournaments = """
query GetTournaments($first: Int, $after: String) {
  tournaments(first: $first, after: $after) {
    pageInfo {
      hasPreviousPage
      hasNextPage
      startCursor
      endCursor
    }
    totalCount
    edges {
      cursor
      node {
        id
        name
        nameShortened
      }
    }
  }
}
"""

def fetch_tournaments(first: int = 20, after: str | None = None):
    payload = {
        "query": querytournaments,
        "operationName": "GetTournaments",
        "variables": {"first": first, "after": after},
    }
    r = requests.post(url_graphql, headers=headers1, json=payload, timeout=30)
    out = r.json()

    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
    if "errors" in out:
        raise RuntimeError(f"GraphQL errors: {json.dumps(out['errors'], indent=2)[:2000]}")

    return out

def tournaments_to_df(out_json: dict) -> pd.DataFrame:
    edges = out_json["data"]["tournaments"]["edges"]
    rows = [{
        "cursor": e["cursor"],
        "tournament_id": e["node"]["id"],
        "name": e["node"]["name"],
        "nameShortened": e["node"]["nameShortened"],
    } for e in edges]
    return pd.DataFrame(rows)


data1 = fetch_tournaments(first=20, after=None)
end_cursor_1 = data1["data"]["tournaments"]["pageInfo"]["endCursor"]
print("page1 endCursor:", end_cursor_1)

data2 = fetch_tournaments(first=20, after=end_cursor_1)
end_cursor_2 = data2["data"]["tournaments"]["pageInfo"]["endCursor"]
print("page2 endCursor:", end_cursor_2)

df1 = tournaments_to_df(data1)
df2 = tournaments_to_df(data2)

df_tournaments = (
    pd.concat([df1, df2], ignore_index=True)
      .drop_duplicates(subset=["tournament_id"])
      .reset_index(drop=True)
)


df_valorant = df_tournaments[
    df_tournaments["name"].str.contains("VCT|VALORANT", case=False, na=False)
].copy()

print("Rows (combined):", len(df_tournaments))
print("Rows (VALORANT filtered):", len(df_valorant))


df_tournaments.head(10)
df_valorant[["tournament_id", "name", "nameShortened"]].reset_index(drop=True)


page1 endCursor: PjMLC1oLC0BCQENPRQsLWgsL
page2 endCursor: PjMLC1oLC0BCT0dAQwsLWgsL
Rows (combined): 40
Rows (VALORANT filtered): 19


,tournament_id,name,nameShortened
0,757073,VCT Americas - Kickoff 2024 (Groups: Group A),Group A
1,757074,VCT Americas - Kickoff 2024 (Playoffs: Playoffs),Playoffs
2,757101,VCT Americas - Kickoff 2024 (Play-Ins: Play In...,Play In Groups
3,757234,VCT Masters - Masters Madrid (Swiss: Swiss),Swiss
4,757235,VALORANT Masters - Masters Madrid (Playoffs: P...,Playoffs
5,757320,VCT Americas - Stage 1 2024 (Regular Season: O...,Omega Group
6,757321,VCT Americas - Stage 1 2024 (Playoffs: Playoffs),Playoffs
7,757371,VCT Americas - Kickoff 2024,Kickoff 2024
8,757372,VCT Americas - Kickoff 2024 (Groups),Groups
9,757481,VCT Americas - Stage 1 2024,Stage 1 2024


<h1>Series</h1> 

In [1]:
import requests
import json
import pandas as pd


url_graphql = "https://api-op.grid.gg/central-data/graphql"


API_KEY = "gDVqIdMead2zTW1DKehu8PicvVXStT2xtmbYHK7b"

headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json",
}


queryseries = """
query GetValorantSeriesList($first: Int, $after: String) {
  allSeries(
    first: $first
    after: $after
    filter: { titleId: "6" }
    orderBy: ID
    orderDirection: ASC
  ) {
    pageInfo {
      hasPreviousPage
      hasNextPage
      startCursor
      endCursor
    }
    totalCount
    edges {
      cursor
      node {
        ...seriesFields
      }
    }
  }
}

query GetValorantSeriesListNextPage($after: String) {
  allSeries(
    first: 20
    after: $after
    filter: { titleId: "6" }
    orderBy: ID
    orderDirection: ASC
  ) {
    pageInfo {
      hasPreviousPage
      hasNextPage
      startCursor
      endCursor
    }
    totalCount
    edges {
      cursor
      node {
        ...seriesFields
      }
    }
  }
}

fragment seriesFields on Series {
  id
  startTimeScheduled
  type
  updatedAt
  title { id name }
  tournament { id name nameShortened }
}

"""

def run_gql(operation_name: str, variables: dict):
    payload = {
        "query": queryseries,
        "operationName": operation_name,
        "variables": variables,
    }
    r = requests.post(url_graphql, headers=headers, json=payload, timeout=30)
    out = r.json()

   
    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
    if "errors" in out:
        print(f" {operation_name} returned errors:")
        print(json.dumps(out["errors"], indent=2)[:2000])
        return out

    return out


data1 = run_gql("GetValorantSeriesList", {"first": 20, "after": None})
all_series_1 = data1["data"]["allSeries"]

end_cursor = all_series_1["pageInfo"]["endCursor"]
print("page1 hasNextPage:", all_series_1["pageInfo"]["hasNextPage"])
print("page1 endCursor:", end_cursor)
print("page1 items:", len(all_series_1["edges"]))


data2 = run_gql("GetValorantSeriesListNextPage", {"after": end_cursor})
all_series_2 = data2["data"]["allSeries"]

print("page2 hasNextPage:", all_series_2["pageInfo"]["hasNextPage"])
print("page2 endCursor:", all_series_2["pageInfo"]["endCursor"])
print("page2 items:", len(all_series_2["edges"]))


def flatten_edges(edges):
    rows = []
    for e in edges:
        n = e["node"]
        rows.append({
            "cursor": e["cursor"],
            "series_id": n["id"],
            "startTimeScheduled": n["startTimeScheduled"],
            "type": n["type"],
            "updatedAt": n["updatedAt"],
            "title_id": n["title"]["id"],
            "title_name": n["title"]["name"],
            "tournament_id": n["tournament"]["id"],
            "tournament_name": n["tournament"]["name"],
            "tournament_short": n["tournament"]["nameShortened"],
        })
    return rows

rows = flatten_edges(all_series_1["edges"]) + flatten_edges(all_series_2["edges"])
df = pd.DataFrame(rows)

df.head(10)


page1 hasNextPage: True
page1 endCursor: PjMLC1oLC0VBQ09BRUMLC1oLCw==
page1 items: 20
page2 hasNextPage: True
page2 endCursor: PjMLC1oLC0VBQkROQEMLC1oLCw==
page2 items: 20


,cursor,series_id,startTimeScheduled,type,updatedAt,title_id,title_name,tournament_id,tournament_name,tournament_short
0,PjMLC1oLC0VBRU5ETkcLC1oLCw==,2629390,2024-02-16T22:00:00Z,ESPORTS,2025-05-26T11:05:25Z,6,Valorant,757073,VCT Americas - Kickoff 2024 (Groups: Group A),Group A
1,PjMLC1oLC0VBRU5ETkYLC1oLCw==,2629391,2024-02-17T01:00:00Z,ESPORTS,2025-05-26T11:05:25Z,6,Valorant,757073,VCT Americas - Kickoff 2024 (Groups: Group A),Group A
2,PjMLC1oLC0VBRU5ETkULC1oLCw==,2629392,2024-02-17T22:00:00Z,ESPORTS,2025-05-26T11:05:25Z,6,Valorant,757628,VCT Americas - Kickoff 2024 (Groups: Group B),Group B
3,PjMLC1oLC0VBRU5ETkQLC1oLCw==,2629393,2024-02-18T01:00:00Z,ESPORTS,2025-05-26T11:05:25Z,6,Valorant,757628,VCT Americas - Kickoff 2024 (Groups: Group B),Group B
4,PjMLC1oLC0VBRU5ETkMLC1oLCw==,2629394,2024-02-18T22:00:00Z,ESPORTS,2025-05-26T11:05:25Z,6,Valorant,757629,VCT Americas - Kickoff 2024 (Groups: Group C),Group C
5,PjMLC1oLC0VBRU5ETkILC1oLCw==,2629395,2024-02-19T22:00:00Z,ESPORTS,2025-05-26T11:05:25Z,6,Valorant,757073,VCT Americas - Kickoff 2024 (Groups: Group A),Group A
6,PjMLC1oLC0VBRU5ETkELC1oLCw==,2629396,2024-02-20T01:00:00Z,ESPORTS,2025-05-26T11:05:25Z,6,Valorant,757628,VCT Americas - Kickoff 2024 (Groups: Group B),Group B
7,PjMLC1oLC0VBRU5ETk8LC1oLCw==,2629398,2024-02-23T22:00:00Z,ESPORTS,2025-05-26T11:05:25Z,6,Valorant,757073,VCT Americas - Kickoff 2024 (Groups: Group A),Group A
8,PjMLC1oLC0VBRU5ETk4LC1oLCw==,2629399,2024-02-24T01:00:00Z,ESPORTS,2025-05-26T11:05:25Z,6,Valorant,757628,VCT Americas - Kickoff 2024 (Groups: Group B),Group B
9,PjMLC1oLC0VBRU5DR0cLC1oLCw==,2629400,2024-02-24T22:00:00Z,ESPORTS,2025-05-26T11:05:25Z,6,Valorant,757629,VCT Americas - Kickoff 2024 (Groups: Group C),Group C


<h1>Esports Organizations</h1>

In [2]:
import requests, time
import pandas as pd

url = "https://api-op.grid.gg/central-data/graphql"
headers = {"x-api-key":"gDVqIdMead2zTW1DKehu8PicvVXStT2xtmbYHK7b","Content-Type":"application/json","Accept":"application/json"}

query = """
query ($after:String) {
  teams(first: 50, after:$after, filter:{ titleId:"6" }) {
    pageInfo { hasNextPage endCursor }
    edges { node { id name title { name } } }
  }
}
"""

after = None
rows = []

while True:
    r = requests.post(url, headers=headers, json={"query": query, "variables": {"after": after}}, timeout=30)
    out = r.json()
    if "errors" in out:
        print(out["errors"])
        break

    teams = out["data"]["teams"]
    for e in teams["edges"]:
        n = e["node"]
        rows.append((n["id"], n["name"], n["title"]["name"]))

    if not teams["pageInfo"]["hasNextPage"]:
        break
    after = teams["pageInfo"]["endCursor"]
    time.sleep(0.2)

df = pd.DataFrame(rows, columns=["team_id","team_name","title_name"])
print(df["title_name"].value_counts())
print(df.head())


[{'message': 'You have exceeded your rate limit, please try again later', 'path': ['queryValue'], 'extensions': {'errorType': 'UNAVAILABLE', 'errorDetail': 'ENHANCE_YOUR_CALM'}}]
title_name
Valorant    900
Name: count, dtype: int64
  team_id          team_name title_name
0       5              3DMAX   Valorant
1      13    ALTERNATE aTTaX   Valorant
2      24  Ninjas in Pyjamas   Valorant
3      25           Alliance   Valorant
4      29     Gambit Esports   Valorant


<h1>Setup</h1>

In [2]:
import requests
import json
import pandas as pd

url_graphql = "https://api.grid.gg/live-data-feed/series-state/graphql"

API_KEY = "gDVqIdMead2zTW1DKehu8PicvVXStT2xtmbYHK7b"
headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json",
}

def run_gql(query: str, operation_name: str, variables: dict):
    payload = {
        "query": query,
        "operationName": operation_name,
        "variables": variables,
    }
    r = requests.post(url_graphql, headers=headers, json=payload, timeout=30)
    out = r.json()

    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
    if "errors" in out:
        print(f"{operation_name} returned errors:")
        print(json.dumps(out["errors"], indent=2)[:2000])
        return out

    return out

SERIES_ID = "2629395"  


<h1>SeriesTeamStateValorant</h1>

In [15]:
import requests
import json
import pandas as pd

url_graphql = "https://api-op.grid.gg/live-data-feed/series-state/graphql"

API_KEY = "gDVqIdMead2zTW1DKehu8PicvVXStT2xtmbYHK7b"  
headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json",
}

query_series_team_state = """
query GetSeriesTeamStateValorant($seriesId: ID!) {
  seriesState(id: $seriesId) {
    id
    teams {
      __typename
      ... on SeriesTeamStateValorant {
        id
        name
        score
        won
        kills
        deaths
        headshots
        teamHeadshots
        killAssistsReceived
        killAssistsGiven
        killAssistsReceivedFromPlayer { id playerId killAssistsReceived }
        weaponKills { id weaponName count }

        teamkills
        teamkillAssistsReceived
        teamkillAssistsGiven
        teamkillAssistsReceivedFromPlayer { id playerId teamkillAssistsReceived }
        weaponTeamkills { id weaponName count }

        selfkills
        structuresDestroyed
        structuresCaptured
        objectives { id type completionCount }
        multikills { id numberOfKills count }

        players {
          __typename
          ... on SeriesPlayerState {
            id
            name
            participationStatus
            kills
            deaths
            killAssistsReceived
            killAssistsGiven
            killAssistsReceivedFromPlayer { id playerId killAssistsReceived }
            weaponKills { id weaponName count }

            teamkills
            teamkillAssistsReceived
            teamkillAssistsGiven
            teamkillAssistsReceivedFromPlayer { id playerId teamkillAssistsReceived }
            weaponTeamkills { id weaponName count }

            selfkills
            structuresDestroyed
            structuresCaptured
            objectives { id type completionCount }
            multikills { id numberOfKills count }
          }
        }
      }
    }
  }
}
"""

def run_gql(operation_name: str, variables: dict):
    payload = {
        "query": query_series_team_state,
        "operationName": operation_name,
        "variables": variables,
    }
    r = requests.post(url_graphql, headers=headers, json=payload, timeout=30)
    out = r.json()

    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
    if "errors" in out:
        print(f"{operation_name} returned errors:")
        print(json.dumps(out["errors"], indent=2)[:2000])
        return out

    return out



def flatten_teams(series_id, teams):
    rows = []
    for t in teams:
        if t.get("__typename") and t["__typename"] != "SeriesTeamStateValorant":
            continue
        rows.append({
            "series_id": series_id,
            "team_id": t.get("id"),
            "team_name": t.get("name"),
            "score": t.get("score"),
            "won": t.get("won"),
            "kills": t.get("kills"),
            "deaths": t.get("deaths"),
            "headshots": t.get("headshots"),
            "teamHeadshots": t.get("teamHeadshots"),
            "killAssistsReceived": t.get("killAssistsReceived"),
            "killAssistsGiven": t.get("killAssistsGiven"),
            "teamkills": t.get("teamkills"),
            "teamkillAssistsReceived": t.get("teamkillAssistsReceived"),
            "teamkillAssistsGiven": t.get("teamkillAssistsGiven"),
            "selfkills": t.get("selfkills"),
            "structuresDestroyed": t.get("structuresDestroyed"),
            "structuresCaptured": t.get("structuresCaptured"),
        })
    return rows

def flatten_players(series_id, teams):
    rows = []
    for t in teams:
        if t.get("__typename") and t["__typename"] != "SeriesTeamStateValorant":
            continue
        team_id = t.get("id")
        for p in (t.get("players") or []):
            rows.append({
                "series_id": series_id,
                "team_id": team_id,
                "player_state_id": p.get("id"),
                "player_name": p.get("name"),
                "participationStatus": p.get("participationStatus"),
                "kills": p.get("kills"),
                "deaths": p.get("deaths"),
                "killAssistsReceived": p.get("killAssistsReceived"),
                "killAssistsGiven": p.get("killAssistsGiven"),
                "teamkills": p.get("teamkills"),
                "teamkillAssistsReceived": p.get("teamkillAssistsReceived"),
                "teamkillAssistsGiven": p.get("teamkillAssistsGiven"),
                "selfkills": p.get("selfkills"),
                "structuresDestroyed": p.get("structuresDestroyed"),
                "structuresCaptured": p.get("structuresCaptured"),
            })
    return rows

def flatten_weapon_kills(series_id, teams):
    rows = []
    for t in teams:
        if t.get("__typename") and t["__typename"] != "SeriesTeamStateValorant":
            continue
        team_id = t.get("id")

      
        for wk in (t.get("weaponKills") or []):
            rows.append({
                "series_id": series_id,
                "scope": "team",
                "team_id": team_id,
                "player_state_id": None,
                "weaponName": wk.get("weaponName"),
                "count": wk.get("count"),
            })

        
        for p in (t.get("players") or []):
            pid = p.get("id")
            for wk in (p.get("weaponKills") or []):
                rows.append({
                    "series_id": series_id,
                    "scope": "player",
                    "team_id": team_id,
                    "player_state_id": pid,
                    "weaponName": wk.get("weaponName"),
                    "count": wk.get("count"),
                })
    return rows

def flatten_objectives(series_id, teams):
    rows = []
    for t in teams:
        if t.get("__typename") and t["__typename"] != "SeriesTeamStateValorant":
            continue
        team_id = t.get("id")

        for o in (t.get("objectives") or []):
            rows.append({
                "series_id": series_id,
                "scope": "team",
                "team_id": team_id,
                "player_state_id": None,
                "objective_id": o.get("id"),
                "type": o.get("type"),
                "completionCount": o.get("completionCount"),
            })

        for p in (t.get("players") or []):
            pid = p.get("id")
            for o in (p.get("objectives") or []):
                rows.append({
                    "series_id": series_id,
                    "scope": "player",
                    "team_id": team_id,
                    "player_state_id": pid,
                    "objective_id": o.get("id"),
                    "type": o.get("type"),
                    "completionCount": o.get("completionCount"),
                })
    return rows

def flatten_multikills(series_id, teams):
    rows = []
    for t in teams:
        if t.get("__typename") and t["__typename"] != "SeriesTeamStateValorant":
            continue
        team_id = t.get("id")

        for m in (t.get("multikills") or []):
            rows.append({
                "series_id": series_id,
                "scope": "team",
                "team_id": team_id,
                "player_state_id": None,
                "numberOfKills": m.get("numberOfKills"),
                "count": m.get("count"),
            })

        for p in (t.get("players") or []):
            pid = p.get("id")
            for m in (p.get("multikills") or []):
                rows.append({
                    "series_id": series_id,
                    "scope": "player",
                    "team_id": team_id,
                    "player_state_id": pid,
                    "numberOfKills": m.get("numberOfKills"),
                    "count": m.get("count"),
                })
    return rows

def to_jsonl(df: pd.DataFrame, path: str):
    df.to_json(path, orient="records", lines=True, force_ascii=False)



series_id = 2629395  

out = run_gql("GetSeriesTeamStateValorant", {"seriesId": series_id})
series = out.get("data", {}).get("seriesState") or {}
sid = series.get("id")
teams = series.get("teams") or []


df_teams = pd.DataFrame(flatten_teams(sid, teams))
df_players = pd.DataFrame(flatten_players(sid, teams))
df_weapon_kills = pd.DataFrame(flatten_weapon_kills(sid, teams))
df_objectives = pd.DataFrame(flatten_objectives(sid, teams))
df_multikills = pd.DataFrame(flatten_multikills(sid, teams))

print("teams:", df_teams.shape)
print("players:", df_players.shape)
print("weapon_kills:", df_weapon_kills.shape)
print("objectives:", df_objectives.shape)
print("multikills:", df_multikills.shape)


import os
output_dir = os.path.expanduser("~/valorant_data")
os.makedirs(output_dir, exist_ok=True)

df_teams.to_csv(os.path.join(output_dir, "teams.csv"), index=False)
df_players.to_csv(os.path.join(output_dir, "players.csv"), index=False)
df_weapon_kills.to_csv(os.path.join(output_dir, "weapon_kills.csv"), index=False)
df_objectives.to_csv(os.path.join(output_dir, "objectives.csv"), index=False)
df_multikills.to_csv(os.path.join(output_dir, "multikills.csv"), index=False)


to_jsonl(df_players, os.path.join(output_dir, "players.jsonl"))
to_jsonl(df_teams, os.path.join(output_dir, "teams.jsonl"))


with open(os.path.join(output_dir, "raw_series_team_state.json"), "w", encoding="utf-8") as f:
    json.dump(out, f, ensure_ascii=False, indent=2)

df_players.head(10)


   series_id  team_id  player_state_id player_name participationStatus  kills  \
0    2629395       81             1814  rglmeister              active     28   
1    2629395       81               59         frz              active     37   
2    2629395       81              266       mazin              active     19   
3    2629395       81             2641         jzz              active     36   
4    2629395       81             2837      artziN              active     37   
5    2629395       99               45      liazzi              active     23   
6    2629395       99               57      mwzera              active     38   
7    2629395       99               19       kon4n              active     25   
8    2629395       99             3314       havoc              active     36   
9    2629395       99               51      Khalil              active     25   

   deaths  killAssistsReceived  killAssistsGiven  teamkills  
0      26                   10                

<h1>SeriesPlayerStateValorant</h1>

In [16]:
import requests
import pandas as pd
import json

url_graphql = "https://api-op.grid.gg/live-data-feed/series-state/graphql"

API_KEY = "gDVqIdMead2zTW1DKehu8PicvVXStT2xtmbYHK7b"
headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json",
}

query_series_team = """
query SeriesTeamStateValorant($seriesId: ID!) {
  seriesState(id: $seriesId) {
    id
    teams {
      __typename
      ... on SeriesTeamStateValorant {
        id
        name
        players {
          __typename
          ... on SeriesPlayerState {
            id
            name
            participationStatus
            kills
            deaths
            killAssistsReceived
            killAssistsGiven
            teamkills
          }
        }
      }
    }
  }
}
"""

def run_gql(query: str, operation_name: str, variables: dict):
    payload = {
        "query": query,
        "operationName": operation_name,
        "variables": variables,
    }
    r = requests.post(url_graphql, headers=headers, json=payload, timeout=30)
    r.raise_for_status()
    out = r.json()
    if "errors" in out:
        raise RuntimeError(json.dumps(out["errors"], indent=2))
    return out

def build_player_dataframe(series_state: dict) -> pd.DataFrame:
    series_id = series_state.get("id")
    teams = series_state.get("teams") or []

    rows = []
    for t in teams:
        team_id = t.get("id")
        players = t.get("players") or []
        for p in players:
            rows.append({
                "series_id": int(series_id) if series_id is not None else None,
                "team_id": team_id,
                "player_state_id": p.get("id"),
                "player_name": p.get("name"),
                "participationStatus": p.get("participationStatus"),
                "kills": p.get("kills"),
                "deaths": p.get("deaths"),
                "killAssistsReceived": p.get("killAssistsReceived"),
                "killAssistsGiven": p.get("killAssistsGiven"),
                "teamkills": p.get("teamkills"),
            })

    df = pd.DataFrame(rows)

    for c in ["series_id", "team_id", "player_state_id", "kills", "deaths",
              "killAssistsReceived", "killAssistsGiven", "teamkills"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    return df


series_id = "2629395"
out = run_gql(query_series_team, "SeriesTeamStateValorant", {"seriesId": series_id})

series_state = (out.get("data") or {}).get("seriesState") or {}
df_players = build_player_dataframe(series_state)

print(df_players.head(10))
print("shape:", df_players.shape)


   series_id  team_id  player_state_id player_name participationStatus  kills  \
0    2629395       81             1814  rglmeister              active     28   
1    2629395       81               59         frz              active     37   
2    2629395       81              266       mazin              active     19   
3    2629395       81             2641         jzz              active     36   
4    2629395       81             2837      artziN              active     37   
5    2629395       99               45      liazzi              active     23   
6    2629395       99               57      mwzera              active     38   
7    2629395       99               19       kon4n              active     25   
8    2629395       99             3314       havoc              active     36   
9    2629395       99               51      Khalil              active     25   

   deaths  killAssistsReceived  killAssistsGiven  teamkills  
0      26                   10                

<h1>GameTeamStateValorant</h1>

In [26]:
import requests
import pandas as pd
import json

url_graphql = "https://api-op.grid.gg/live-data-feed/series-state/graphql"

API_KEY = "gDVqIdMead2zTW1DKehu8PicvVXStT2xtmbYHK7b"
headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json",
}

query_game_team = """
query GameTeamStateValorant($seriesId: ID!) {
  seriesState(id: $seriesId) {
    id
    games {
      id
      teams {
        __typename
        ... on GameTeamStateValorant {
          id
          name
          side
          won
          score
          players {
            id
            name
            character { id name }
            participationStatus
            money
            loadoutValue
            netWorth
            kills
            deaths
            killAssistsReceived
            killAssistsGiven
            teamkills
            selfkills
            structuresDestroyed
            structuresCaptured
          }
        }
      }
    }
  }
}
"""

def run_gql(query: str, operation_name: str, variables: dict):
    payload = {"query": query, "operationName": operation_name, "variables": variables}
    r = requests.post(url_graphql, headers=headers, json=payload, timeout=30)
    r.raise_for_status()
    out = r.json()
    if "errors" in out:
        raise RuntimeError(json.dumps(out["errors"], indent=2))
    return out

def build_game_player_dataframe(series_state: dict) -> pd.DataFrame:
    series_id = series_state.get("id")
    games = series_state.get("games") or []

    rows = []
    for g in games:
        game_id = g.get("id")
        for t in (g.get("teams") or []):
            if t.get("__typename") and t["__typename"] != "GameTeamStateValorant":
                continue

            team_id = t.get("id")
            team_name = t.get("name")
            side = t.get("side")
            team_score = t.get("score")
            team_won = t.get("won")

            for p in (t.get("players") or []):
                rows.append({
                    "series_id": str(series_id) if series_id is not None else None,
                    "game_id": str(game_id) if game_id is not None else None,
                    "team_id": str(team_id) if team_id is not None else None,
                    "team_name": team_name,
                    "side": side,
                    "team_score": team_score,
                    "team_won": team_won,
                    "player_id": str(p.get("id")) if p.get("id") is not None else None,
                    "player_name": p.get("name"),
                    "character": (p.get("character") or {}).get("name"),
                    "participationStatus": p.get("participationStatus"),
                    "money": p.get("money"),
                    "loadoutValue": p.get("loadoutValue"),
                    "netWorth": p.get("netWorth"),
                    "kills": p.get("kills"),
                    "deaths": p.get("deaths"),
                    "killAssistsReceived": p.get("killAssistsReceived"),
                    "killAssistsGiven": p.get("killAssistsGiven"),
                    "teamkills": p.get("teamkills"),
                    "selfkills": p.get("selfkills"),
                    "structuresDestroyed": p.get("structuresDestroyed"),
                    "structuresCaptured": p.get("structuresCaptured"),
                })

    df = pd.DataFrame(rows)

    stat_cols = [
        "team_score", "money", "loadoutValue", "netWorth",
        "kills", "deaths", "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills", "structuresDestroyed", "structuresCaptured"
    ]
    for c in stat_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    if "kills" in df.columns and "deaths" in df.columns:
        df["kd_ratio"] = df["kills"] / df["deaths"].replace(0, 1)

    cols = [
        "series_id","game_id","team_id","team_name","side","team_won","team_score",
        "player_id","player_name","character","participationStatus",
        "kills","deaths","kd_ratio","killAssistsReceived","killAssistsGiven","teamkills",
        "money","loadoutValue","netWorth","selfkills","structuresDestroyed","structuresCaptured"
    ]
    cols = [c for c in cols if c in df.columns]
    df = df[cols + [c for c in df.columns if c not in cols]]

    return df

series_id = "2629395"
out = run_gql(query_game_team, "GameTeamStateValorant", {"seriesId": series_id})
series_state = (out.get("data") or {}).get("seriesState") or {}
df_game_players = build_game_player_dataframe(series_state)

print(df_game_players.head(10))
print("shape:", df_game_players.shape)


  series_id                               game_id team_id team_name      side  \
0   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      81      MIBR  attacker   
1   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      81      MIBR  attacker   
2   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      81      MIBR  attacker   
3   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      81      MIBR  attacker   
4   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      81      MIBR  attacker   
5   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      99     FURIA  defender   
6   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      99     FURIA  defender   
7   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      99     FURIA  defender   
8   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      99     FURIA  defender   
9   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      99     FURIA  defender   

   team_won  team_score player_id player_name character  ...  kd_ratio  \
0      True          13      1814 

<h1>GamePlayerStateValorant</h1>

In [27]:
import requests
import json
import pandas as pd
import os

url_graphql = "https://api-op.grid.gg/live-data-feed/series-state/graphql"
API_KEY = os.getenv("GRID_API_KEY") or "gDVqIdMead2zTW1DKehu8PicvVXStT2xtmbYHK7b"

headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json",
}

query_game_player = """
query GamePlayerStateValorant($seriesId: ID!) {
  seriesState(id: $seriesId) {
    id
    games {
      id
      teams {
        players {
          __typename
          ... on GamePlayerStateValorant {
            id
            name
            character { id name }
            participationStatus
            money
            loadoutValue
            netWorth
            kills
            deaths
            headshots
            teamHeadshots
            killAssistsReceived
            killAssistsGiven
            killAssistsReceivedFromPlayer { id playerId killAssistsReceived }
            weaponKills { id weaponName count }
            teamkills
            teamkillAssistsReceived
            teamkillAssistsGiven
            teamkillAssistsReceivedFromPlayer { id playerId teamkillAssistsReceived }
            weaponTeamkills { id weaponName count }
            selfkills
            structuresDestroyed
            structuresCaptured
            inventory { items { id quantity equipped stashed } }
            objectives { id type completionCount }
            position { x y }
            multikills { id numberOfKills count }
            unitKills { id unitName count }
            abilities { id name ready }
            alive
            currentArmor
            currentHealth
            maxHealth
          }
        }
      }
    }
  }
}
"""

def run_gql(query: str, operation_name: str, variables: dict) -> dict:
    payload = {"query": query, "operationName": operation_name, "variables": variables}
    r = requests.post(url_graphql, headers=headers, json=payload, timeout=30)
    try:
        out = r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON response. HTTP {r.status_code}: {r.text[:500]}")
    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
    if "errors" in out:
        raise RuntimeError(json.dumps(out["errors"], indent=2)[:4000])
    return out

def build_game_players_df(out: dict) -> pd.DataFrame:
    series_state = (out.get("data") or {}).get("seriesState") or {}
    series_id = series_state.get("id")
    games = series_state.get("games") or []

    rows = []
    keep_scalar = [
        "participationStatus",
        "money", "loadoutValue", "netWorth",
        "kills", "deaths", "headshots", "teamHeadshots",
        "teamkills", "selfkills",
        "structuresDestroyed", "structuresCaptured",
        "alive", "currentArmor", "currentHealth", "maxHealth",
        "killAssistsReceived", "killAssistsGiven",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
    ]

    for g in games:
        game_id = g.get("id")
        for t in (g.get("teams") or []):
            for p in (t.get("players") or []):
                if p.get("__typename") and p["__typename"] != "GamePlayerStateValorant":
                    continue

                rows.append({
                    "series_id": str(series_id) if series_id is not None else None,
                    "game_id": str(game_id) if game_id is not None else None,
                    "player_id": str(p.get("id")) if p.get("id") is not None else None,
                    "player_name": p.get("name"),
                    "character": (p.get("character") or {}).get("name"),
                    **{k: p.get(k) for k in keep_scalar}
                })

    df = pd.DataFrame(rows)

    stat_cols = [
        "money", "loadoutValue", "netWorth",
        "kills", "deaths", "headshots", "teamHeadshots",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
        "structuresDestroyed", "structuresCaptured",
        "currentArmor", "currentHealth", "maxHealth",
    ]
    for c in stat_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    if "alive" in df.columns:
        df["alive"] = df["alive"].astype("boolean")

    if "kills" in df.columns and "deaths" in df.columns:
        df["kd_ratio"] = df["kills"] / df["deaths"].replace(0, 1)

    preferred_order = [
        "series_id", "game_id",
        "player_id", "player_name", "character", "participationStatus",
        "kills", "deaths", "headshots",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills",
        "money", "loadoutValue", "netWorth",
        "alive", "currentArmor", "currentHealth", "maxHealth",
        "kd_ratio",
    ]
    existing = [c for c in preferred_order if c in df.columns]
    remaining = [c for c in df.columns if c not in existing]
    return df[existing + remaining]

series_id = "2629395"
out = run_gql(query_game_player, "GamePlayerStateValorant", {"seriesId": series_id})
df_game_players = build_game_players_df(out)

print("type:", type(df_game_players))
print("shape:", df_game_players.shape)
print(df_game_players.head(10))


type: <class 'pandas.DataFrame'>
shape: (20, 26)
  series_id                               game_id player_id player_name  \
0   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      1814  rglmeister   
1   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce        59         frz   
2   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce       266       mazin   
3   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      2641         jzz   
4   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      2837      artziN   
5   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce        45      liazzi   
6   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce        57      mwzera   
7   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce        19       kon4n   
8   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce      3314       havoc   
9   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce        51      Khalil   

  character participationStatus  kills  deaths  headshots  \
0     astra              active     11      13         12   
1  

<h1>SegmentTeamStateValorant</h1>

In [28]:
import requests
import json
import pandas as pd
import os

url_graphql = "https://api-op.grid.gg/live-data-feed/series-state/graphql"

API_KEY = os.getenv("GRID_API_KEY") or "gDVqIdMead2zTW1DKehu8PicvVXStT2xtmbYHK7b"

headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json",
}

query_segment_team = """
query SegmentTeamStateValorantQuery($seriesId: ID!) {
  seriesState(id: $seriesId) {
    id
    games {
      id
      segments {
        id
        teams {
          __typename
          ... on SegmentTeamStateValorant {
            id
            name
            side
            won
            kills
            killAssistsReceived
            killAssistsGiven
            killAssistsReceivedFromPlayer { id playerId killAssistsReceived }
            weaponKills { id weaponName count }
            teamkills
            teamkillAssistsReceived
            teamkillAssistsGiven
            teamkillAssistsReceivedFromPlayer { id playerId teamkillAssistsReceived }
            weaponTeamkills { id weaponName count }
            selfkills
            deaths
            objectives { id type completionCount }
            players {
              __typename
              ... on SegmentPlayerState {
                id
                name
                kills
                killAssistsReceived
                killAssistsGiven
                killAssistsReceivedFromPlayer { id playerId killAssistsReceived }
                weaponKills { id weaponName count }
                teamkills
                teamkillAssistsReceived
                teamkillAssistsGiven
                teamkillAssistsReceivedFromPlayer { id playerId teamkillAssistsReceived }
                weaponTeamkills { id weaponName count }
                selfkills
                deaths
                objectives { id type completionCount }
              }
            }
            winType
            headshots
            teamHeadshots
          }
        }
      }
    }
  }
}
"""

def run_gql(query: str, operation_name: str, variables: dict) -> dict:
    payload = {"query": query, "operationName": operation_name, "variables": variables}
    r = requests.post(url_graphql, headers=headers, json=payload, timeout=30)
    try:
        out = r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON response. HTTP {r.status_code}: {r.text[:500]}")
    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
    if "errors" in out:
        raise RuntimeError(json.dumps(out["errors"], indent=2)[:4000])
    return out

def build_segment_teams_df(out: dict) -> pd.DataFrame:
    series_state = (out.get("data") or {}).get("seriesState") or {}
    series_id = series_state.get("id")
    games = series_state.get("games") or []

    rows = []
    keep_team_scalar = [
        "kills", "deaths",
        "headshots", "teamHeadshots",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
        "winType",
    ]

    for g in games:
        game_id = g.get("id")
        for seg in (g.get("segments") or []):
            segment_id = seg.get("id")
            for t in (seg.get("teams") or []):
                if t.get("__typename") and t["__typename"] != "SegmentTeamStateValorant":
                    continue

                row = {
                    "series_id": str(series_id) if series_id is not None else None,
                    "game_id": str(game_id) if game_id is not None else None,
                    "segment_id": str(segment_id) if segment_id is not None else None,
                    "segment_team_id": str(t.get("id")) if t.get("id") is not None else None,
                    "team_name": t.get("name"),
                    "side": t.get("side"),
                    "won": t.get("won"),
                }

                for k in keep_team_scalar:
                    row[k] = t.get(k)

                rows.append(row)

    df = pd.DataFrame(rows)

    stat_cols = [
        "kills", "deaths", "headshots", "teamHeadshots",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
    ]
    for c in stat_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    if "won" in df.columns:
        df["won"] = df["won"].astype("boolean")

    preferred = [
        "series_id", "game_id", "segment_id",
        "segment_team_id", "team_name", "side",
        "won", "winType",
        "kills", "deaths", "headshots", "teamHeadshots",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
    ]
    existing = [c for c in preferred if c in df.columns]
    remaining = [c for c in df.columns if c not in existing]
    return df[existing + remaining]

def build_segment_players_df(out: dict) -> pd.DataFrame:
    series_state = (out.get("data") or {}).get("seriesState") or {}
    series_id = series_state.get("id")
    games = series_state.get("games") or []

    rows = []
    keep_player_scalar = [
        "kills", "deaths",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
    ]

    for g in games:
        game_id = g.get("id")
        for seg in (g.get("segments") or []):
            segment_id = seg.get("id")
            for t in (seg.get("teams") or []):
                if t.get("__typename") and t["__typename"] != "SegmentTeamStateValorant":
                    continue

                segment_team_id = t.get("id")
                team_name = t.get("name")
                side = t.get("side")

                for p in (t.get("players") or []):
                    if p.get("__typename") and p["__typename"] != "SegmentPlayerState":
                        continue

                    row = {
                        "series_id": str(series_id) if series_id is not None else None,
                        "game_id": str(game_id) if game_id is not None else None,
                        "segment_id": str(segment_id) if segment_id is not None else None,
                        "segment_team_id": str(segment_team_id) if segment_team_id is not None else None,
                        "team_name": team_name,
                        "side": side,
                        "segment_player_id": str(p.get("id")) if p.get("id") is not None else None,
                        "player_name": p.get("name"),
                    }

                    for k in keep_player_scalar:
                        row[k] = p.get(k)

                    rows.append(row)

    df = pd.DataFrame(rows)

    stat_cols = [
        "kills", "deaths",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
    ]
    for c in stat_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    if "kills" in df.columns and "deaths" in df.columns:
        df["kd_ratio"] = df["kills"] / df["deaths"].replace(0, 1)

    preferred = [
        "series_id", "game_id", "segment_id",
        "segment_team_id", "team_name", "side",
        "segment_player_id", "player_name",
        "kills", "deaths", "kd_ratio",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
    ]
    existing = [c for c in preferred if c in df.columns]
    remaining = [c for c in df.columns if c not in existing]
    return df[existing + remaining]

series_id = "2629395"
out = run_gql(query_segment_team, "SegmentTeamStateValorantQuery", {"seriesId": series_id})

df_segment_teams = build_segment_teams_df(out)
df_segment_players = build_segment_players_df(out)

print("Segment teams:", df_segment_teams.shape)
print(df_segment_teams.head(10))

print("\nSegment players:", df_segment_players.shape)
print(df_segment_players.head(10))


Segment teams: (92, 18)
  series_id                               game_id segment_id segment_team_id  \
0   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1              81   
1   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1              99   
2   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-2              81   
3   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-2              99   
4   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-3              81   
5   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-3              99   
6   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-4              81   
7   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-4              99   
8   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-5              81   
9   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-5              99   

  team_name      side    won             winType  kills  deaths  headshots  \
0      MIBR  defe

<h1>SegmentPlayerStateValorant</h1>

In [23]:
import requests
import json
import pandas as pd
import os


url_graphql = "https://api-op.grid.gg/live-data-feed/series-state/graphql"

API_KEY =  "gDVqIdMead2zTW1DKehu8PicvVXStT2xtmbYHK7b"

headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json",
    "Accept": "application/json",
}


query_segment_player = """
query SegmentPlayerStateValorant($seriesId: ID!) {
  seriesState(id: $seriesId) {
    id
    games {
      id
      segments {
        id
        teams {
          __typename
          ... on SegmentTeamStateValorant {
            id
            name
            side
            won
            players {
              __typename
              ... on SegmentPlayerStateValorant {
                id
                name
                kills
                deaths
                killAssistsReceived
                killAssistsGiven
                killAssistsReceivedFromPlayer { id playerId killAssistsReceived }
                weaponKills { id weaponName count }
                teamkills
                teamkillAssistsReceived
                teamkillAssistsGiven
                teamkillAssistsReceivedFromPlayer { id playerId teamkillAssistsReceived }
                weaponTeamkills { id weaponName count }
                selfkills
                objectives { id type completionCount }
                alive
                currentArmor
                currentHealth
                maxHealth
                headshots
                teamHeadshots
              }
            }
          }
        }
      }
    }
  }
}
"""


def run_gql(query: str, operation_name: str, variables: dict) -> dict:
    payload = {"query": query, "operationName": operation_name, "variables": variables}
    r = requests.post(url_graphql, headers=headers, json=payload, timeout=30)

    try:
        out = r.json()
    except Exception:
        raise RuntimeError(f"Non-JSON response. HTTP {r.status_code}: {r.text[:500]}")

    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:500]}")
    if "errors" in out:
        raise RuntimeError(json.dumps(out["errors"], indent=2)[:4000])

    return out


def iter_segment_players(series_state: dict):
    """Yield (series_id, game_id, segment_id, team_info_dict, player_dict)"""
    series_id = series_state.get("id")
    for g in (series_state.get("games") or []):
        game_id = g.get("id")
        for seg in (g.get("segments") or []):
            segment_id = seg.get("id")
            for t in (seg.get("teams") or []):
                if t.get("__typename") and t["__typename"] != "SegmentTeamStateValorant":
                    continue
                team_info = {
                    "team_id": t.get("id"),
                    "team_name": t.get("name"),
                    "side": t.get("side"),
                    "won": t.get("won"),
                }
                for p in (t.get("players") or []):
                    if p.get("__typename") and p["__typename"] != "SegmentPlayerStateValorant":
                        continue
                    yield series_id, game_id, segment_id, team_info, p


def build_segment_players_df(series_state: dict) -> pd.DataFrame:
    rows = []
    keep_scalar = [
        "kills", "deaths",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
        "alive", "currentArmor", "currentHealth", "maxHealth",
        "headshots", "teamHeadshots",
    ]

    for series_id, game_id, segment_id, team_info, p in iter_segment_players(series_state):
        row = {
            "series_id": str(series_id) if series_id is not None else None,
            "game_id": str(game_id) if game_id is not None else None,
            "segment_id": str(segment_id) if segment_id is not None else None,
            **{k: (str(v) if v is not None else None) for k, v in team_info.items() if k in ["team_id"]},
            "team_name": team_info.get("team_name"),
            "side": team_info.get("side"),
            "won": team_info.get("won"),
            "segment_player_id": str(p.get("id")) if p.get("id") is not None else None,
            "player_name": p.get("name"),
        }
        for k in keep_scalar:
            row[k] = p.get(k)
        rows.append(row)

    df = pd.DataFrame(rows)

    stat_cols = [
        "kills", "deaths",
        "headshots", "teamHeadshots",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
        "currentArmor", "currentHealth", "maxHealth",
    ]
    for c in stat_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

 
    if "alive" in df.columns:
        df["alive"] = df["alive"].astype("boolean")
    if "won" in df.columns:
        df["won"] = df["won"].astype("boolean")

  
    if "kills" in df.columns and "deaths" in df.columns:
        df["kd_ratio"] = df["kills"] / df["deaths"].replace(0, 1)

   
    preferred = [
        "series_id", "game_id", "segment_id",
        "team_id", "team_name", "side", "won",
        "segment_player_id", "player_name",
        "kills", "deaths", "kd_ratio",
        "headshots", "teamHeadshots",
        "killAssistsReceived", "killAssistsGiven",
        "teamkills", "selfkills",
        "teamkillAssistsReceived", "teamkillAssistsGiven",
        "alive", "currentArmor", "currentHealth", "maxHealth",
    ]
    existing = [c for c in preferred if c in df.columns]
    remaining = [c for c in df.columns if c not in existing]
    df = df[existing + remaining]

    return df


def build_segment_weapon_kills_df(series_state: dict) -> pd.DataFrame:
    rows = []
    for series_id, game_id, segment_id, team_info, p in iter_segment_players(series_state):
        for wk in (p.get("weaponKills") or []):
            rows.append({
                "series_id": str(series_id) if series_id is not None else None,
                "game_id": str(game_id) if game_id is not None else None,
                "segment_id": str(segment_id) if segment_id is not None else None,
                "team_id": str(team_info.get("team_id")) if team_info.get("team_id") is not None else None,
                "team_name": team_info.get("team_name"),
                "side": team_info.get("side"),
                "segment_player_id": str(p.get("id")) if p.get("id") is not None else None,
                "player_name": p.get("name"),
                "weapon_name": wk.get("weaponName"),
                "count": wk.get("count"),
            })

    df = pd.DataFrame(rows)
    if "count" in df.columns:
        df["count"] = pd.to_numeric(df["count"], errors="coerce")
    return df

def build_segment_objectives_df(series_state: dict) -> pd.DataFrame:
    rows = []
    for series_id, game_id, segment_id, team_info, p in iter_segment_players(series_state):
        for obj in (p.get("objectives") or []):
            rows.append({
                "series_id": str(series_id) if series_id is not None else None,
                "game_id": str(game_id) if game_id is not None else None,
                "segment_id": str(segment_id) if segment_id is not None else None,
                "team_id": str(team_info.get("team_id")) if team_info.get("team_id") is not None else None,
                "team_name": team_info.get("team_name"),
                "side": team_info.get("side"),
                "segment_player_id": str(p.get("id")) if p.get("id") is not None else None,
                "player_name": p.get("name"),
                "objective_type": obj.get("type"),
                "completionCount": obj.get("completionCount"),
            })

    df = pd.DataFrame(rows)
    if "completionCount" in df.columns:
        df["completionCount"] = pd.to_numeric(df["completionCount"], errors="coerce")
    return df


def add_aggregated_features(df_players: pd.DataFrame,
                            df_weapon_kills: pd.DataFrame,
                            df_objectives: pd.DataFrame) -> pd.DataFrame:
    key = ["series_id", "game_id", "segment_id", "segment_player_id"]

 
    if not df_weapon_kills.empty:
        w_tot = (df_weapon_kills.groupby(key, as_index=False)["count"]
                 .sum()
                 .rename(columns={"count": "weapon_kills_total"}))
        w_wide = (df_weapon_kills
                  .pivot_table(index=key, columns="weapon_name", values="count", aggfunc="sum", fill_value=0)
                  .reset_index()
                  .rename(columns=lambda c: c if c in key else f"wk_{c}"))
    else:
        w_tot = pd.DataFrame(columns=key + ["weapon_kills_total"])
        w_wide = pd.DataFrame(columns=key)

 
    if not df_objectives.empty:
        o_tot = (df_objectives.groupby(key, as_index=False)["completionCount"]
                 .sum()
                 .rename(columns={"completionCount": "objectives_total"}))
        o_wide = (df_objectives
                  .pivot_table(index=key, columns="objective_type", values="completionCount", aggfunc="sum", fill_value=0)
                  .reset_index()
                  .rename(columns=lambda c: c if c in key else f"obj_{c}"))
    else:
        o_tot = pd.DataFrame(columns=key + ["objectives_total"])
        o_wide = pd.DataFrame(columns=key)

    out = (df_players
           .merge(w_tot, on=key, how="left")
           .merge(w_wide, on=key, how="left")
           .merge(o_tot, on=key, how="left")
           .merge(o_wide, on=key, how="left"))


    for col in out.columns:
        if col == "weapon_kills_total" or col == "objectives_total" or col.startswith(("wk_", "obj_")):
            out[col] = out[col].fillna(0)

    return out


series_id = "2629395"
out = run_gql(query_segment_player, "SegmentPlayerStateValorant", {"seriesId": series_id})
series_state = (out.get("data") or {}).get("seriesState") or {}

df_segment_players = build_segment_players_df(series_state)
df_segment_weapon_kills = build_segment_weapon_kills_df(series_state)
df_segment_objectives = build_segment_objectives_df(series_state)

df_features = add_aggregated_features(df_segment_players, df_segment_weapon_kills, df_segment_objectives)

print("players:", df_segment_players.shape)
print(df_segment_players.head(10))

print("\nweaponKills (long):", df_segment_weapon_kills.shape)
print(df_segment_weapon_kills.head(10))

print("\nobjectives (long):", df_segment_objectives.shape)
print(df_segment_objectives.head(10))

print("\nfeatures joined:", df_features.shape)
print(df_features.head(10))



players: (460, 24)
  series_id                               game_id segment_id team_id  \
0   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1      81   
1   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1      81   
2   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1      81   
3   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1      81   
4   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1      81   
5   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1      99   
6   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1      99   
7   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1      99   
8   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1      99   
9   2629395  2cd86d08-a06b-4c0f-b749-a0f2964a68ce    round-1      99   

  team_name      side    won segment_player_id player_name  kills  ...  \
0      MIBR  defender   True              1814  rglmeister      0  ...   
1      MIBR  defender   True            